# Classification Model using BQML deploying to Vertex AI (Manual) 

## Environment Set up
This demo uses ecommerce Google Analytics data, publicly available as a BigQuery dataset. 

In [1]:
PROJECT_ID = 'hivedemoindia' #replace value
LOCATION = 'us-central1'
MODEL_NAME = 'web_analytics_clsmod'
DATASET_NAME= 'webAnalytics'

BUCKET_NAME = f'{PROJECT_ID}_{MODEL_NAME}'
DATASET_ID = f'{PROJECT_ID}.{DATASET_NAME}'

In [2]:
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
new_bucket = storage_client.create_bucket(bucket, location=LOCATION)

Conflict: 409 POST https://storage.googleapis.com/storage/v1/b?project=hivedemoindia&prettyPrint=false: Your previous request to create the named bucket succeeded and you already own it.

## Create Database 

In [3]:
from google.cloud import bigquery

client = bigquery.Client()
dataset = bigquery.Dataset(DATASET_ID)
dataset.location = "US"
dataset = client.create_dataset(dataset, timeout=30) 

print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/hivedemoindia/datasets?prettyPrint=false: Already Exists: Dataset hivedemoindia:webAnalytics

## Explore Data

In [4]:
from google.cloud import bigquery
query_job = client.query(
    """
WITH visitors AS(
  SELECT
    COUNT(DISTINCT fullVisitorId) AS total_visitors
  FROM
    `data-to-insights.ecommerce.web_analytics` ),
  purchasers AS(
  SELECT
    COUNT(DISTINCT fullVisitorId) AS total_purchasers
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.transactions IS NOT NULL )
SELECT
  total_visitors,
  total_purchasers,
  total_purchasers / total_visitors AS conversion_rate
FROM
  visitors,
  purchasers"""
)

results = query_job.result().to_dataframe()
print(results.head())

   total_visitors  total_purchasers  conversion_rate
0          741721             20015         0.026985


In [5]:
# visitors who bought on a return visit (could have bought on first as well
query_job = client.query(
    """
WITH all_visitor_stats AS (
    SELECT
      fullvisitorid, 
      IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
    FROM `data-to-insights.ecommerce.web_analytics`
    GROUP BY fullvisitorid
    )
SELECT
    COUNT(DISTINCT fullvisitorid) AS total_visitors,
      will_buy_on_return_visit
    FROM all_visitor_stats
    GROUP BY will_buy_on_return_visit"""
)

results = query_job.result().to_dataframe()
print(results.head(5))

   total_visitors  will_buy_on_return_visit
0          729848                         0
1           11873                         1


## Create BQML Classification Model
Predict if visitors "will buy in future" or "won't buy in future", using logistic_reg in a classification model.

In [6]:
from google.cloud import bigquery
client = bigquery.Client()

model_build = f"""
CREATE OR REPLACE MODEL {DATASET_NAME}.{MODEL_NAME}
OPTIONS(
    model_type='logistic_reg',
    labels = ['will_buy_on_return_visit']
    )
AS
SELECT
  * EXCEPT(fullVisitorId)
FROM
  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1
    AND date BETWEEN '20160801' AND '20170430') # train on first 9 months
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId);
"""

query_job = client.query(model_build)
query_state = client.get_job(query_job.job_id, location=query_job.location).state

print(f'Query: {query_state}')

while query_state == 'RUNNING': 
    query_state = client.get_job(query_job.job_id, location=query_job.location).state
print(f'Query: {query_state} \n * Note: DONE does not ensure sucessful completion - check UI')

Query: RUNNING
Query: DONE 
 * Note: DONE does not ensure sucessful completion - check UI


## Evaluation 

In [7]:
from google.cloud import bigquery
query_job = client.query(f"""
SELECT
  roc_auc,
  CASE
    WHEN roc_auc > .9 THEN 'good'
    WHEN roc_auc > .8 THEN 'fair'
    WHEN roc_auc > .7 THEN 'decent'
    WHEN roc_auc > .6 THEN 'not great'
  ELSE 'poor' END AS model_quality
FROM
  ML.EVALUATE(MODEL {DATASET_NAME}.{MODEL_NAME},  (
SELECT
  * EXCEPT(fullVisitorId)
FROM
  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1
    AND date BETWEEN '20170501' AND '20170630') # eval on 2 months
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
));""")
results = query_job.result().to_dataframe()

print(results.head(5))

    roc_auc model_quality
0  0.723839        decent


## Export Model to Vertex AI 
**Export to GCS then upload to Vertex AI**

In [8]:
!bq extract -m {PROJECT_ID}:{DATASET_NAME}.{MODEL_NAME} gs://{BUCKET_NAME}/{MODEL_NAME}

Waiting on bqjob_r6624221a0d004763_00000183ac3f4111_1 ... (25s) Current status: DONE   


**Upload model to Vertex**

In [9]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)
model = aiplatform.Model.upload(
        display_name=MODEL_NAME,
        artifact_uri= f'gs://{BUCKET_NAME}/{MODEL_NAME}',
        serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest' ) 
model.wait()

print(model.display_name)
print(model.resource_name)

Creating Model
Create Model backing LRO: projects/688817582093/locations/us-central1/models/8737928857098649600/operations/669723871192023040
Model created. Resource name: projects/688817582093/locations/us-central1/models/8737928857098649600@1
To use this Model in another session:
model = aiplatform.Model('projects/688817582093/locations/us-central1/models/8737928857098649600@1')
web_analytics_clsmod
projects/688817582093/locations/us-central1/models/8737928857098649600


**Deploy to an endpoint**

In [ ]:
endpoint = aiplatform.Endpoint.create( display_name=f'{MODEL_NAME}_endpt', project=PROJECT_ID, location=LOCATION)

model.deploy(
        endpoint=endpoint,
        traffic_percentage = 100,
        machine_type ='n1-highcpu-2')

model.wait()
print(model.resource_name)

Creating Endpoint
Create Endpoint backing LRO: projects/688817582093/locations/us-central1/endpoints/1306892714914086912/operations/8123744204443615232
Endpoint created. Resource name: projects/688817582093/locations/us-central1/endpoints/1306892714914086912
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/688817582093/locations/us-central1/endpoints/1306892714914086912')
Deploying model to Endpoint : projects/688817582093/locations/us-central1/endpoints/1306892714914086912
Deploy Endpoint model backing LRO: projects/688817582093/locations/us-central1/endpoints/1306892714914086912/operations/5405258879372099584


## Predict
Predict if the visitors will buy on return of visit using the API 

In [4]:
%%writefile default-pred.json
{"instances": [{"bounces" :0,"time_on_site":60}, {"bounces" :0,"time_on_site": 18102}]}

Writing default-pred.json


In [5]:
ENDPOINT_ID="1306892714914086912"
#ENDPOINT_ID=endpoint.resource_name

!curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/$PROJECT_ID/locations/us-central1/endpoints/$ENDPOINT_ID:predict \
-d "@default-pred.json"

{
  "predictions": [
    {
      "predicted_will_buy_on_return_visit": [
        "0"
      ],
      "will_buy_on_return_visit_probs": [
        0.02274886132318673,
        0.97725113867681324
      ],
      "will_buy_on_return_visit_values": [
        "1",
        "0"
      ]
    },
    {
      "predicted_will_buy_on_return_visit": [
        "1"
      ],
      "will_buy_on_return_visit_values": [
        "1",
        "0"
      ],
      "will_buy_on_return_visit_probs": [
        0.99999050416474577,
        9.495835254225149e-06
      ]
    }
  ],
  "deployedModelId": "8989242030836154368",
  "model": "projects/688817582093/locations/us-central1/models/8737928857098649600",
  "modelDisplayName": "web_analytics_clsmod",
  "modelVersionId": "1"
}


# Creating a Vertex AI Pipeline

In [ ]:
Go to bqpipeline_demo.ipynb

In [ ]:
#resources: https://www.qwiklabs.com/focuses/1794?parent=catalog
#https://cloud.google.com/bigquery-ml/docs/exporting-models?_ga=2.59990958.-2027684164.1621380090
#https://docs.google.com/document/d/1wre9hLVx-H8syG-806UPWGJbDVGieIM5VvFKi8lbGtw/edit
